In [89]:
import numpy as np
import pandas as pd
import math



In [90]:
import tensorflow as tf 

In [91]:
from tensorflow.keras import backend as K

In [92]:
df = pd.read_csv('C:\TPC_GAN_SIM\TPC-GAN-SIM\Data\digits.csv')

In [93]:
df.describe()

,evtId,ipad,itime,amp,crossing_angle,dip_angle,drift_length,pad_coordinate
count,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000,544513.000000
mean,9998.116271,43.191593,172.717893,243.400471,-0.057631,-0.505002,172.718675,43.191624
std,5767.839085,1.540539,72.647436,639.426251,11.693803,36.664225,72.613729,1.152069
min,0.000000,39.000000,31.000000,1.000000,-20.000000,-59.985000,35.878000,41.192000
25%,5011.000000,42.000000,111.000000,7.362000,-10.357000,-33.817000,110.722000,42.207000
50%,10002.000000,43.000000,177.000000,46.470000,-0.066000,-0.654000,177.216000,43.175000
75%,14987.000000,44.000000,236.000000,260.600000,10.201000,32.684000,236.080000,44.194000
max,19999.000000,48.000000,296.000000,39990.000000,20.099000,59.993000,290.405000,45.192000


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544513 entries, 0 to 544512
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   evtId           544513 non-null  int64  
 1   ipad            544513 non-null  int64  
 2   itime           544513 non-null  int64  
 3   amp             544513 non-null  float64
 4   crossing_angle  544513 non-null  float64
 5   dip_angle       544513 non-null  float64
 6   drift_length    544513 non-null  float64
 7   pad_coordinate  544513 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 33.2 MB


In [95]:
input_features = df.iloc[:,-4:]
input_features

,crossing_angle,dip_angle,drift_length,pad_coordinate
0,-8.695,32.201,35.936,41.844
1,-8.695,32.201,35.936,41.844
2,-8.695,32.201,35.936,41.844
3,-8.695,32.201,35.936,41.844
4,-8.695,32.201,35.936,41.844
...,...,...,...,...
544508,10.016,-12.861,52.403,42.317
544509,10.016,-12.861,52.403,42.317
544510,10.016,-12.861,52.403,42.317
544511,10.016,-12.861,52.403,42.317


In [96]:
def linearity_scaling(pd_series ,  a = -1, b = 1):
    x,y = pd_series.min() , pd_series.max()
    out = (pd_series - x) /(y-x) *(b-a) +a
    
    return out 
    

In [97]:
def preprocessing_func ( features):
    
    temp_features = pd.DataFrame()
    #it is sufficient to only feed the fractional part of the pad coordinate into our model
    temp_features['pad_coordinate'] = features['pad_coordinate'] % 1 
    
    # drift length both the fractional part and the full number are fed into the model as two separate features
    
    temp_features ['drift_length_frac'] = features['drift_length'] % 1
    
    
    
    # the angles and the drift length are linearly scaled down to a [−1, 1] region
    
    columns = ['crossing_angle','dip_angle','drift_length']
    
    for col in columns:
        temp_features[col] = linearity_scaling(features[col])
    
    
    return temp_features

In [98]:
tmp_df = preprocessing_func(input_features)
tmp_df.head()

,pad_coordinate,drift_length_frac,crossing_angle,dip_angle,drift_length
0,0.844,0.936,-0.436146,0.536715,-0.999544
1,0.844,0.936,-0.436146,0.536715,-0.999544
2,0.844,0.936,-0.436146,0.536715,-0.999544
3,0.844,0.936,-0.436146,0.536715,-0.999544
4,0.844,0.936,-0.436146,0.536715,-0.999544


In [99]:
tmp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 544513 entries, 0 to 544512
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   pad_coordinate     544513 non-null  float64
 1   drift_length_frac  544513 non-null  float64
 2   crossing_angle     544513 non-null  float64
 3   dip_angle          544513 non-null  float64
 4   drift_length       544513 non-null  float64
dtypes: float64(5)
memory usage: 20.8 MB


In [100]:
preprocessed_input_features = tf.convert_to_tensor(tmp_df)
print (preprocessed_input_features)

tf.Tensor(
[[ 0.844       0.936      -0.43614554  0.53671506 -0.99954425]
 [ 0.844       0.936      -0.43614554  0.53671506 -0.99954425]
 [ 0.844       0.936      -0.43614554  0.53671506 -0.99954425]
 ...
 [ 0.317       0.403       0.49709469 -0.21445598 -0.8701513 ]
 [ 0.317       0.403       0.49709469 -0.21445598 -0.8701513 ]
 [ 0.317       0.403       0.49709469 -0.21445598 -0.8701513 ]], shape=(544513, 5), dtype=float64)


In [101]:

'''
custom log with base in tensorflow if there was a problem with the values
def log10(x):
  numerator = tf.log(x)
  denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
  return numerator / denominator
'''

def custom_activation(x , T = math.log10(2), alpha = 0.1, gamma = 0.01 ):
    '''
    if gamma < x :
        T-gamma +x
    elif 0<x<=gamma:
        T * (alpha + ((1- alpha)*(x/gamma)))
    else :
        alpha * T * math.exp(x)
    
    '''
    return K.switch(x> gamma , T - gamma + x , K.switch(x < 0, alpha * T * tf.math.exp(x),T * (alpha + ((1- alpha)*(x/gamma)))))

In [102]:
#testing the activation function 

X = np.random.uniform(0,1, (100,10))
y = np.random.uniform(0,1, 100)

inp = tf.keras.Input((10,))
x = tf.keras.layers.Dense(8, activation=custom_activation)(inp)
out = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inp, out)
model.compile('adam', 'mse')
model.fit(X,y, epochs=3)

Epoch 1/3
4/4 [==============================] - 0s 3ms/step - loss: 0.1431
Epoch 2/3
4/4 [==============================] - 0s 4ms/step - loss: 0.1308
Epoch 3/3
4/4 [==============================] - 0s 4ms/step - loss: 0.1192


In [103]:
#Appendix A: Generator


# attention, care that last dense layer function should be more described 
def create_generator_structure():
    
    generator = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(64,activation='relu'),
            tf.keras.layers.Dense(64,activation='relu'),
            tf.keras.layers.Dense(64,activation='relu'),
            tf.keras.layers.Dense(8*16,activation=custom_activation),
            tf.keras.layers.Reshape(target_shape= (8,16))
        ],
        name= 'generator'
    )
    return generator

In [104]:
model = create_generator_structure()
model.build(input_shape= (37,1))
model.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_77 (Dense)            (37, 32)                  64        
                                                                 
 dense_78 (Dense)            (37, 64)                  2112      
                                                                 
 dense_79 (Dense)            (37, 64)                  4160      
                                                                 
 dense_80 (Dense)            (37, 64)                  4160      
                                                                 
 dense_81 (Dense)            (37, 128)                 8320      
                                                                 
 reshape_14 (Reshape)        (37, 8, 16)               0         
                                                                 
Total params: 18,816
Trainable params: 18,816
Non-trainab

In [122]:
#Appendix A: discriminator

def create_discriminator_structure():
    input_img = tf.keras.Input(shape= (8,16))
    features = tf.keras.Input(shape= (5,))    
    
    # reshaping 
    
    # remarks : If one component of shape is the special value -1, the size of that dimension is computed so that the total size remains constant. In particular, a shape of [-1] flattens into 1-D. At most one component of shape can be -1.
    
    
    img = tf.reshape(input_img, (-1,8,16,1))
    
    
    features_Tiled = tf.tile((tf.reshape(features, (-1,1,1,5))), (1,8,16,1))
    
    # remarks : Negative axis are interpreted as counting from the end of the rank, i.e., axis + rank(values)-th dimension.
    # remarks : The rank of a tensor is the number of indices required to uniquely select each element of the tensor.

    input_img = tf.concat([img,features_Tiled],-1) 
    
    
    
    
    
    discriminator_main = tf.keras.Sequential(
        [
            tf.keras.layers.Conv2D(activation='relu', kernel_size=(3,3),filters= 16 , padding='same'),
            tf.keras.layers.Dropout(0.02),
            
            tf.keras.layers.Conv2D(activation='relu',kernel_size= (3,3),filters=16, padding='same'),
            tf.keras.layers.Dropout(0.02),
            
            tf.keras.layers.MaxPool2D(pool_size=(1,2)),
            
            tf.keras.layers.Conv2D(activation='relu', kernel_size=(3,3),filters= 32 , padding='same'),
            tf.keras.layers.Dropout(0.02),
            
            tf.keras.layers.Conv2D(activation='relu',kernel_size= (3,3),filters=32, padding='same'),
            tf.keras.layers.Dropout(0.02),
            
            tf.keras.layers.MaxPool2D(pool_size=(2,2)),
            
            tf.keras.layers.Conv2D(activation='relu', kernel_size=(3,3),filters= 64 , padding='valid'),
            tf.keras.layers.Dropout(0.02),
            
            tf.keras.layers.Conv2D(activation='relu',kernel_size= (2,2),filters=64, padding='valid'),
            tf.keras.layers.Dropout(0.02),
            
            tf.keras.layers.Reshape(target_shape=(64,))
        ],
        name= 'discriminator_main'
    )
    
    head_input  = tf.keras.layers.Concatenate()([features,discriminator_main(input_img)])
    
    head_layers  = [
        tf.keras.layers.Dense(128,activation='relu'),
        tf.keras.layers.Dropout(0.02),
        
        
        tf.keras.layers.Dense(1,activation=None)
    ]
    
    discriminator_head = tf.keras.Sequential(
        head_layers,
        name = 'discriminator_head'
    )
    
    inputs = [features, input_img]
    outputs = discriminator_head(head_input)
    
    discriminator = tf.keras.Model(inputs=inputs, outputs=outputs, name='discriminator')

    return discriminator
    
    
     

In [134]:
model = create_discriminator_structure()
model.build(input_shape=(1,8,16,1))
model.summary()

Model: "discriminator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 8, 16, 6)]   0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, 5)]          0           []                               
                                                                                                  
 discriminator_main (Sequential  (None, 64)          52032       ['input_21[0][0]']               
 )                                                                                                
                                                                                                  
 concatenate_4 (Concatenate)    (None, 69)           0           ['input_20[0][0]',   